In [0]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import  LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

Using TensorFlow backend.


Upload files using upload button in the left pane.

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Dataset-Categorywise/training.csv')
test_df = pd.read_csv('/content/drive/My Drive/Dataset-Categorywise/testing.csv')

In [0]:
train_df.shape,test_df.shape

((1435, 4), (354, 4))

In [0]:
train_df = train_df[~train_df.Article.isnull()]
test_df =  test_df[~test_df.Article.isnull()]

In [0]:
train_df.shape,test_df.shape

((1430, 4), (353, 4))

In [0]:
train_df.head(1)

,Category,Date_and_Time,Keywords,Article
0,Entertainment,"\nAugust 23, 2018 16:58 IST\n",\nBring on the laughs,"A dash of stand up comedy, lots of improvisat..."


In [0]:
train_df.Category.value_counts()

Food & Health            160
Business & Economy       160
Education & Career       160
International            160
Politics & Governance    160
Others                   160
Sports                   160
Science & Technology     156
Entertainment            154
Name: Category, dtype: int64

In [0]:
le=LabelEncoder()
le.fit(train_df.Category.values)

LabelEncoder()

In [0]:
X_train = train_df[['Article']]
X_test =  test_df[['Article']]
y_train = train_df['Category']
y_test = test_df['Category']

In [0]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [0]:
# X_train.Article = X_train.Article.str.lower()
# X_test.Article  = X_test.Article.str.lower()
train_sentences = X_train.Article.values
test_sentences= X_test.Article.values

# # We need to add special tokens at the beginning and end of each sentence for BERT to work properly
# train_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in train_sentences]
# test_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in test_sentences]

**PreProcessing** for XLNET

In [0]:
train_sentences = [sentence + " [SEP] [CLS]" for sentence in train_sentences]
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]

In [0]:
!pip install pytorch_transformers
from pytorch_transformers import *

     |████████████████████████████████| 184kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 10.6MB/s 
     |████████████████████████████████| 870kB 19.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=2c3b22d099803dbecdd0cbc1b6e605cc3c7c269a604a3de2c0a4b01e45e325db
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
%%time
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
train_tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
test_tokenized_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
print ("Tokenize the first sentence:")
print (train_tokenized_texts[0])


100%|██████████| 798011/798011 [00:00<00:00, 4602875.69B/s]


Tokenize the first sentence:
['▁A', '▁dash', '▁of', '▁stand', '▁up', '▁comedy', ',', '▁lots', '▁of', '▁improvisation', ',', '▁and', '▁a', '▁', 'ton', '▁of', '▁audience', '▁interaction', '.', '▁These', '▁are', '▁what', '▁the', '▁performers', '▁have', '▁promised', '▁to', '▁give', '▁Bengal', 'uru', '▁tonight', '▁at', '▁a', '▁', 'mar', 'que', 'e', '▁show', ',', '▁organised', '▁as', '▁part', '▁of', '▁Black', '▁Dog', '▁Easy', '▁Evening', 's', '.', '▁The', '▁event', '▁brings', '▁together', '▁the', '▁talents', '▁of', '▁Papa', '▁C', 'J', ',', '▁Cyrus', '▁Sa', 'hu', 'kar', ',', '▁Danish', '▁Sai', 't', '▁and', '▁The', '▁Improv', ',', '▁hosted', '▁by', '▁Saad', '▁Khan', '.', '▁After', '▁shows', '▁in', '▁Delhi', '▁and', '▁Hyderabad', ',', '▁the', '▁performers', '▁are', '▁in', '▁our', '▁city', '▁tonight', '▁and', '▁the', '▁act', '▁promises', '▁much', '▁variety', ',', '▁with', '▁a', '▁stand', '▁up', '▁set', '▁by', '▁Papa', '▁C', 'J', '▁followed', '▁by', '▁improv', '▁comedy', '▁by', '▁The', '▁Improv',

In [0]:
train_input_ids=[]
for i in tqdm_notebook(range(len(train_tokenized_texts))):
  train_input_ids.append(tokenizer.convert_tokens_to_ids(train_tokenized_texts[i]))


In [0]:
test_input_ids=[]
for i in tqdm_notebook(range(len(test_tokenized_texts))):
  test_input_ids.append(tokenizer.convert_tokens_to_ids(test_tokenized_texts[i]))


In [0]:
##Padding 
MAX_LEN = 512
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
#Create attention masks
train_attention_masks = []
test_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in train_input_ids:
  seq_mask = [float(i>0) for i in seq]
  train_attention_masks.append(seq_mask)
  
# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

In [0]:
train_inputs = torch.tensor(train_input_ids)
validation_inputs = torch.tensor(test_input_ids)
train_labels = torch.tensor(y_train)
validation_labels = torch.tensor(y_test)
train_masks = torch.tensor(train_attention_masks)
validation_masks = torch.tensor(test_attention_masks)

In [0]:
batch_size = 8

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device ='cpu'
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased',num_labels=le.classes_.shape[0])
model.to(device)

100%|██████████| 467042463/467042463 [00:08<00:00, 52576668.34B/s]


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [0]:
# lr = 2e-5
# max_grad_norm = 1.0
# num_total_steps = 1000
# num_warmup_steps = 100
# warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1


# ### In PyTorch-Transformers, optimizer and schedules are splitted and instantiated like this:
# optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)  # PyTorch scheduler
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [0]:
model_wts='/content/drive/My Drive/News_claasifiers/XLNET' ## in our drive wew will make a folder to save model weights

In [0]:
total_step = len(train_dataloader)

# Store our loss and accuracy for plotting
train_loss_set = []


epochs = 15
val_accuracy=0
# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)):
  
  

    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs[0]
      train_loss_set.append(loss.item())    
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      # scheduler.step()
      optimizer.zero_grad()
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))
      # Test the model
    with torch.no_grad():
        correct = 0
        total = 0
        for i, batch in enumerate(validation_dataloader):
          batch = tuple(t.to(device) for t in batch)
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          # print (outputs)
          prediction = torch.argmax(outputs[0],dim=1)
          total += b_labels.size(0)
          correct+=(prediction==b_labels).sum().item()
    curr__val_accuracy=100 * correct / total
    if curr__val_accuracy>val_accuracy:
      val_accuracy=curr__val_accuracy
      torch.save(model.state_dict(), model_wts+'/model_without_language_model.ckpt')
      tokenizer.save_pretrained(model_wts+'/Data')
      print ("Model saved")
    print('Validation accuracy {} {}'.format(epoch,100 * correct / total))  

Epoch [1/15], Step [1/179], Loss: 2.2791
Epoch [1/15], Step [51/179], Loss: 1.9829
Epoch [1/15], Step [101/179], Loss: 1.0534
Epoch [1/15], Step [151/179], Loss: 0.7886


Saving directory (/content/drive/My Drive/News_claasifiers/XLNET/Data) should be a directory


Model saved
Validation accuracy 0 80.1699716713881
Epoch [2/15], Step [1/179], Loss: 0.1748
Epoch [2/15], Step [51/179], Loss: 0.5896
Epoch [2/15], Step [101/179], Loss: 0.3884
Epoch [2/15], Step [151/179], Loss: 0.1925


Saving directory (/content/drive/My Drive/News_claasifiers/XLNET/Data) should be a directory


Model saved
Validation accuracy 1 81.86968838526913
Epoch [3/15], Step [1/179], Loss: 0.0455
Epoch [3/15], Step [51/179], Loss: 0.6123
Epoch [3/15], Step [101/179], Loss: 0.5255
Epoch [3/15], Step [151/179], Loss: 0.4600


...

Model saved
Validation accuracy 2 83.28611898016997
Epoch [4/15], Step [1/179], Loss: 0.5816
Epoch [4/15], Step [51/179], Loss: 0.0241
Epoch [4/15], Step [101/179], Loss: 0.0180
Epoch [4/15], Step [151/179], Loss: 0.0181
Validation accuracy 3 79.3201133144476
Epoch [5/15], Step [1/179], Loss: 0.0488
Epoch [5/15], Step [51/179], Loss: 0.0193
Epoch [5/15], Step [101/179], Loss: 0.0146
Epoch [5/15], Step [151/179], Loss: 0.0466
Validation accuracy 4 78.47025495750708
Epoch [6/15], Step [1/179], Loss: 0.0106
Epoch [6/15], Step [51/179], Loss: 0.0330
Epoch [6/15], Step [101/179], Loss: 0.3817
Epoch [6/15], Step [151/179], Loss: 0.0189
Validation accuracy 5 79.3201133144476
Epoch [7/15], Step [1/179], Loss: 0.0452
Epoch [7/15], Step [51/179], Loss: 0.0637
Epoch [7/15], Step [101/179], Loss: 0.0091
Epoch [7/15], Step [151/179], Loss: 0.0965
Validation accuracy 6 82.43626062322946
Epoch [8/15], Step [1/179], Loss: 0.0277
Epoch [8/15], Step [51/179], Loss: 0.0056
Epoch [8/15], Step [101/179], L

Saving directory (/content/drive/My Drive/News_claasifiers/XLNET/Data) should be a directory


Model saved
Validation accuracy 9 83.56940509915015
Epoch [11/15], Step [1/179], Loss: 0.0011
Epoch [11/15], Step [51/179], Loss: 0.0030
Epoch [11/15], Step [101/179], Loss: 0.0013
Epoch [11/15], Step [151/179], Loss: 0.0032
Validation accuracy 10 82.43626062322946
Epoch [12/15], Step [1/179], Loss: 0.0015
Epoch [12/15], Step [51/179], Loss: 0.0032
Epoch [12/15], Step [101/179], Loss: 0.0012
Epoch [12/15], Step [151/179], Loss: 0.0012


Saving directory (/content/drive/My Drive/News_claasifiers/XLNET/Data) should be a directory


Model saved
Validation accuracy 11 84.13597733711048
Epoch [13/15], Step [1/179], Loss: 0.0011
Epoch [13/15], Step [51/179], Loss: 0.0011
Epoch [13/15], Step [101/179], Loss: 0.0013
Epoch [13/15], Step [151/179], Loss: 0.0012
Validation accuracy 12 81.86968838526913
Epoch [14/15], Step [1/179], Loss: 0.0012
Epoch [14/15], Step [51/179], Loss: 0.0010
Epoch [14/15], Step [101/179], Loss: 0.0014
Epoch [14/15], Step [151/179], Loss: 0.0012
Validation accuracy 13 83.28611898016997
Epoch [15/15], Step [1/179], Loss: 0.0011
Epoch [15/15], Step [51/179], Loss: 0.0010
Epoch [15/15], Step [101/179], Loss: 0.0010
Epoch [15/15], Step [151/179], Loss: 0.0009
Validation accuracy 14 83.28611898016997



In [0]:
os.listdir(model_wts)


['model_without_language_model.ckpt']

In [0]:
saved_model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased',num_labels=le.classes_.shape[0])
saved_model.load_state_dict(torch.load(model_wts+"/model_without_language_model.ckpt"))
saved_model.eval()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [0]:
test_predictions=[]
for i, batch in enumerate(validation_dataloader):
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = saved_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          # print (outputs)
          prediction = torch.argmax(outputs[0],dim=1)
          test_predictions.append(prediction)

In [0]:
predictions=[]
for tp in test_predictions:
  predictions.extend(tp.numpy().tolist())

In [0]:
predictions=le.inverse_transform(predictions)

In [0]:
actual_test_labels=le.inverse_transform(y_test)

METRICS ANALYSIS

In [0]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report

In [0]:
##Accuracy 
print ("Accuracy of BERT is {}".format(str(accuracy_score(actual_test_labels,predictions))))


Accuracy of BERT is 0.8243626062322946


In [0]:
### confusion matrix
pd.crosstab(actual_test_labels, predictions, rownames=['Actual'], colnames=['Predicted'])

Predicted,Business & Economy,Education & Career,Entertainment,Food & Health,International,Others,Politics & Governance,Science & Technology,Sports
Actual,,,,,,,,,
Business & Economy,35,0,0,1,0,0,0,4,0
Education & Career,1,32,2,0,0,3,0,2,0
Entertainment,0,1,34,0,0,2,1,1,0
Food & Health,0,0,0,34,1,3,0,2,0
International,2,0,2,0,35,0,1,0,0
Others,1,2,3,0,2,19,7,4,2
Politics & Governance,1,0,0,0,1,4,28,0,0
Science & Technology,1,0,0,1,1,2,0,35,0
Sports,0,1,0,0,0,0,0,0,39


In [0]:
report = classification_report(actual_test_labels, predictions, output_dict=True)


In [0]:
pd.DataFrame(report).T

,precision,recall,f1-score,support
Business & Economy,0.853659,0.875000,0.864198,40.000000
Education & Career,0.888889,0.800000,0.842105,40.000000
Entertainment,0.829268,0.871795,0.850000,39.000000
Food & Health,0.944444,0.850000,0.894737,40.000000
International,0.875000,0.875000,0.875000,40.000000
Others,0.575758,0.475000,0.520548,40.000000
Politics & Governance,0.756757,0.823529,0.788732,34.000000
Science & Technology,0.729167,0.875000,0.795455,40.000000
Sports,0.951220,0.975000,0.962963,40.000000
accuracy,0.824363,0.824363,0.824363,0.824363
